In [1]:
import pandas as pd
from pulp import *

In [2]:
df = pd.read_csv("/Users/ayushjain/Documents/AI-Powered Business Operations & Profit Optimization System/data/business_data.csv")
forecast = pd.read_csv("../data/forecast.csv")
inventory = pd.read_csv("/Users/ayushjain/Documents/AI-Powered Business Operations & Profit Optimization System/data/inventory_plan.csv")

In [3]:
forecast_demand = forecast.groupby('Product')['Forecasted_Demand'].sum().reset_index()
forecast_demand.columns = ['Product','Forecast_Demand']

In [4]:
cost_data = df.groupby('Product').agg({
    'Unit_Cost':'mean',
    'Holding_Cost':'mean',
    'Shortage_Cost':'mean',
    'Capacity':'mean'
}).reset_index()

In [5]:
data = forecast_demand.merge(cost_data, on='Product')

In [6]:
model = LpProblem("Production_Optimization", LpMinimize)

decision_vars = {}

In [7]:
for i, row in data.iterrows():
    product = row['Product']
    decision_vars[product] = LpVariable(f"Q_{product}", lowBound=0)

In [8]:
model += lpSum([
    row['Unit_Cost'] * decision_vars[row['Product']]
    for i, row in data.iterrows()
])

In [9]:
for i, row in data.iterrows():
    product = row['Product']
    model += decision_vars[product] >= row['Forecast_Demand']

In [10]:
for i, row in data.iterrows():
    product = row['Product']
    model += decision_vars[product] <= row['Capacity']

In [11]:
print("Solver Status:", LpStatus[model.solve()])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ayushjain/.local/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/mk/w3ylt_fx2xqdw2h8vpsxdz440000gn/T/12e4540857c9483c85ea6810c66550e1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/mk/w3ylt_fx2xqdw2h8vpsxdz440000gn/T/12e4540857c9483c85ea6810c66550e1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 96 RHS
At line 133 BOUNDS
At line 134 ENDATA
Problem MODEL has 36 rows, 18 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 6109.9141 (18)
17  Obj 36690.971 Primal inf 6007.9141 (18)
Primal infeasible - objective value 36690.971
PrimalInfeasible objective 36690.97065 - 17 iterations

In [12]:
results = []
for product, var in decision_vars.items():
    results.append([product, var.varValue])

In [13]:
production_plan = pd.DataFrame(results, columns=['Product','Optimal_Production'])
production_plan.to_csv("../data/production_plan.csv", index=False)

production_plan.head()

,Product,Optimal_Production
0,Apples,342.50304
1,Bananas,327.75597
2,Bread,373.23352
3,Carrots,281.85006
4,Cereal,348.75089
